# Algoritmo di ricerca output massimo

Il programma cerca entro un numero massimo di tentativi (max_tries) la configurazione del sistema che massimizza un singolo output (connesso al Picoscope), al termine ne salva la configurazione su file. 

## 1) Import

In [1]:
# Required imports
import sys
username = 'admin' # 'admin' for BO1 lab
sys.path.append('c:/users/'+ username +'/miniconda3/lib/site-packages')
path_to_MyCustomPackage = '../' # relative path to the MyCustomPackage folder 
sys.path.append(path_to_MyCustomPackage)
from MyCustomPackage import mycustommodule
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from matplotlib.patches import Polygon
from matplotlib.ticker import MaxNLocator
from cycler import cycler
from IPython.display import display
from IPython.display import clear_output
import qontrol
import os
import time
import numpy as np
import datetime
import os.path
import math
import logging
import wx

%matplotlib widget

app = wx.App(False) # If you get the error The wx.App object must be created first!, uncomment this line


## 2) Dichiarazione variabili e setup driver

In [2]:
# Variable declarations
max_tries = 3
voltage = 12.0
#channels = [4,8,9,11,15,16,17,19,20,21,22,25,26,27] # the channels connected to the device
channels = [4,8,9,17,18,15,19,20,21]
np.random.shuffle(channels)
channel_PD = 3 # the channel connected to the photodiode
voltage_PD = 0 # the voltage to be set to the photodiode
voltage_start = [0.0 for j in range(len(channels))] # starting voltage values for each channel
voltage_stop = [voltage for j in range(len(channels))] # stopping voltage values for each channel
voltage_step = [0.5 for j in range(len(channels))] # voltage steps for each channel
max_current = 10 # current compliance
max_voltage = 12.0 # voltage compliance
measured_voltage = [[[] for i in range(len(channels))] for j in range(len(channels))] # measured_voltage[channels][channel_under_sweep][measurement]
measured_current = [[[] for i in range(len(channels))] for j in range(len(channels))] # current measurements
PD_voltage = [[] for i in range(len(channels))] # PD voltage measurements
PD_current = [[] for i in range(len(channels))] # PD current measurements
F_move_on = 0 # FLAG : 0 for pausing when a channel sweep is done
F_overlapping_plots = 1 # FLAG : 0 for non overlapping plots, not 0 otherwise 
F_triangular = 1 # FLAG : 0 for ramp sweep, 1 for triangular sweep
now = datetime.datetime.now() # retrieve current date and time (for file name)


# Configuration (Laser and Amplifier)
laser_wavelength = 850
laser_power = 4.93
amplifier_gain = 40
device_model = "Q3_+02_+08"
device = device_model + "_" + str(voltage)  + "V_2sweep"


#Create a custom directory
today = str(datetime. date. today()) 
curr_year = today[:4]; curr_month = today[5:7] 
curr_day = today[8:10]
today = datetime.datetime.now()

if today.hour < 10:
    hour = "0" + str(today.hour)
else:
    hour = today.hour
if today.minute < 10:
    minute = "0" + str(today.minute)
else:
    minute = today.minute   
    
# Directory
directory = curr_year + curr_month + curr_day + str(hour) + str(minute)
  
# Parent Directory path
save_path = "G:/Shared drives/FMPS Archive/Projects/0_QPIXPAD/QPX4/Opt.Meas/" + device_model + "/"
  
# Path
path = os.path.join(save_path, directory)
save_path = path

# Create the directory
os.mkdir(path)
print("Directory '% s' created" % directory)


# Set the right path and file

file_name = "Algoritmo_di_ricerca_output_massimo_" + now.strftime("%d_%m_%Y_%H_%M_%S")+ device + '.txt' 
completeName = os.path.join(save_path, file_name)

# Create an ID an initialize it to a number of zeros equal to the maximum length of the total steps
ID = "1"

if F_triangular == 1:
    total_steps = int(((voltage_stop[0]-voltage_start[0])/voltage_step[0])*len(channels)*max_tries*2)
else:
    total_steps = int(((voltage_stop[0]-voltage_start[0])/voltage_step[0])*len(channels)*max_tries)

for i in range(len(str(total_steps))):
    ID = ID + "0"
    
# ------------------ BUILD THE NEW TABLE FOR SAVING CONFIGURATION ---------------------------------

# System configuration will be a list (A) of lists (B). Each element of the list (A) identifies
# an istantneous configuration of the system (stored in a list (B)), composed as:
#        [ID, F_triangular, valore_output_pico, channels[channel_n], I1, V1, ... , In, Vn]

system_configuration = [[] for _ in range(total_steps)]
    

# Build the header of the file as timestamp+configuration+annotation+voltage_parameters+description
timestamp = '%'+now.strftime("%d/%m/%Y %H:%M:%S")
configuration = '\n%Laser Wavelength:' + str(laser_wavelength) + '[nm]\n%Laser Power:' +str(laser_power) + '[mW]\n%Amplifier Gain:' \
              + str(amplifier_gain) + '[dB]'
annotation = '\n%TEST ALGORITMO DI RICERCA OUTPUT MASSIMO. INPUT 5 E OUTPUT 6. TENTATIVO DI MASSIMIZZARE OUTPUT 5.\n'

voltage_parameters = '%' # voltage_parameters are voltage_start, voltage_stop, voltage_step, triangular
for element in voltage_start:
    voltage_parameters += str(element) + '\t'
voltage_parameters += '\n%'
for element in voltage_stop:
    voltage_parameters += str(element) + '\t'
voltage_parameters += '\n%'
for element in voltage_step:
    voltage_parameters += str(element) + '\t'
voltage_parameters += '\n%' + str(F_triangular) + '\n'

description = '%Swipe sulla tensione con onda sinusoidale su un canale, canali rimanenti a 0 V.Ciascun canale pilota una termoresistenza.' + device
header = timestamp+configuration+annotation+voltage_parameters+description

# Write the header and the column headers to the file
meas_file = open(completeName, "a")
meas_file.write(header)
meas_file.write("\n%PD at channel " + str(channel_PD) + ": Voltage [V]\tCurrent [mA]")
for j in range(len(channels)):
    meas_file.write("\tChannel " +str(channels[j]) + " : Voltage [V]\tCurrent [mA]")
meas_file.write("\tTimestamp")
meas_file.write("\n")


# Setup Qontroller
serial_port_name = "COM3" #"COM3" #"/dev/tty.usbserial-FT06QAZ5" # name of the USB Serial Port #//./COM11
q = qontrol.QXOutput(serial_port_name = serial_port_name, response_timeout = 0.1)

# Set the compliances
for channel in range(q.n_chs):
    q.imax[channel] = max_current
    q.vmax[channel] = max_voltage

# Video Debugging
print ("Qontroller '{:}' initialised with firmware {:} and {:} channels".format(q.device_id, q.firmware, q.n_chs) )

Directory '202203101151' created
Qontroller 'Q8b-0471' initialised with firmware v2.4.1 and 32 channels


## 3) Ricerca del massimo

In [ ]:
# Starting Pico with range of 2V, sample interval of 250 microseconds and 5000 total samples
ps = mycustommodule.ps
ctypes = mycustommodule.ctypes
channel_range = ps.PS4000_RANGE['PS4000_5V']
sampleInterval = ctypes.c_int32(250) # in sample units specified by sampleUnits
sampleUnits = ps.PS4000_TIME_UNITS['PS4000_US']
sizeOfOneBuffer = 10 # size of a single buffer
numBuffersToCapture = 1 # totalSamples = sizeOfOneBuffer * numBuffersToCapture
mycustommodule.pico_start(channel_range, sampleInterval = sampleInterval, sampleUnits = sampleUnits, sizeOfOneBuffer = sizeOfOneBuffer, numBuffersToCapture = numBuffersToCapture)

target_configuration = [[0, 0.0, [0.0 for j in range(len(channels))]] for i in range(len(channels))] # list of sublist of two elements. First element is output voltage, second element is a list of all driving voltages for each channel
voltage_max = 0.0
max_attuale = 0.0

current_step = 0


# set all channels to their voltage_start values
for k in range(len(channels)):
    q.v[channels[k]] = float(voltage_start[k])

for i in range(max_tries): # ricerca fino a max_tries e proponi il miglior risultato
    print("\nPERFORMING TRY N.{:} NOW \n" .format(i+1))
    
    #sweep di tensione su ogni canale
    for channel_n in range(len(channels)): # esamina tutti i dof (canali/termoresistenze)
        x = channel_n
        print("\nDRIVING CHANNEL {:} NOW \n" .format(channels[channel_n]))

        #check if we want to generate a triangular sweep or not
        if F_triangular == 1:
            sweep_range = np.concatenate((np.arange(voltage_start[channel_n], voltage_stop[channel_n], voltage_step[channel_n]),\
                            np.arange(voltage_stop[channel_n], voltage_start[channel_n], -voltage_step[channel_n])))
        elif F_triangular == 0:
            sweep_range = np.arange(voltage_start[0], voltage_stop[0]+voltage_step[0], voltage_step[0])
        else:
            print('invalid value for triangular')
            
        for voltage_sweep in sweep_range: 
            q.v[channels[channel_n]] = float(voltage_sweep)
            temp_voltage = q.v[channels[channel_n]]

            valore_output_pico = mycustommodule.pico_acquire_measurement(channel_range, sampleInterval = sampleInterval, sampleUnits = sampleUnits, sizeOfOneBuffer = sizeOfOneBuffer, numBuffersToCapture = numBuffersToCapture);
            
            if valore_output_pico > max_attuale: # se trovi un nuovo massimo, salvalo al posto del massimo finora trovato
                voltage_max = q.v[channels[channel_n]]
                max_attuale = valore_output_pico
                
            # ----- UPDATING THE CONFIGURATION SYSTEM LIST ----------------------
            
            # Creating a fixed length list for first row (for higher performance)
            new_row = [-1 for _ in range((len(channels)*2)+4)]

            # Populate first part of the new row with: ID, flag for triangular sweep, voltage value from pico, active channel
            new_row[0] = int(ID)
            new_row[1] = F_triangular
            new_row[2] = valore_output_pico
            new_row[3] = channels[channel_n]

            # Populate remaining part of the new row with: I1, V1, ..., In, Vn
            for j in range(len(channels)): # acquire voltage and current for all channels. Then print them on screen and save them on file
                temp_voltage = q.v[channels[j]]
                temp_current = q.i[channels[j]]
                new_row[2*j+4] = temp_current
                new_row[2*j+5] = temp_voltage
                
            system_configuration[current_step] = new_row
            current_step += 1
            
            # ---------------------------------------------------------------------
            
            # Take a screenshot of a resolution of screen_resolution and cutting everything that is on the left of xsrc (in pixels) and above ysrc
            #mycustommodule.take_screenshot(xdest= 0, ydest = 0, xsrc = 960, ysrc = 0, image_name = str(ID), save_path = save_path, screen_resolution = [960,1080])
            #mycustommodule.take_screenshot(image_name = str(ID), save_path = save_path)
            ID = int(ID) + 1
            
        q.v[channels[channel_n]] = voltage_max # setta il canale attuale alla tensione corrispondente al suo massimo
        print("\nVoltage maximum for channel {:} found: {:} V" .format(channels[channel_n],voltage_max))
        
        target_configuration[channel_n][0] = int(ID)
        
        target_configuration[channel_n][1] = max_attuale # salva l'output del massimo per il canale attuale
        print("\nCorresponding output from Picoscope: {:} mV" .format(max_attuale))
        
        # Take a screenshot of a resolution of screen_resolution and cutting everything that is on the left of xsrc (in pixels) and above ysrc
        mycustommodule.take_screenshot(xdest= 0, ydest = 0, xsrc = 960, ysrc = 0, image_name = str(ID), save_path = save_path, screen_resolution = [960,1080])
        
        # storing in target_configuration the entire current configuration (corresponding to the current found maximum)
        for k in range(len(channels)):
            target_configuration[channel_n][2][k] = q.v[channels[k]] # salva per il canale attuale, nella lista delle driving voltages, ciascun driving voltage
        
        
        """
        # at the end of each sweep, set all channels to their voltage_start values
        for k in range(len(channels)):
            q.v[channels[k]] = float(voltage_start[k])
        """
"""   commented out and moved in the next section to see the changes on camera before setting all thermistors to 0V     
# Set all channels to zero
q.v[:] = 0
q.i[:] = 0

# Close the communication with the driver
q.close()
"""
# Close the Picoscope
mycustommodule.pico_stop()

print(target_configuration)

# Set the System to last row of target configuration (i.e. the configuration resembling the maximum for the last sweep performed)
print("Setting the system to last row of target configuration")
for k in range(len(target_configuration[-1][2])):
    q.v[channels[k]] = target_configuration[-1][2][k]

def save_whole_system_configuration(system_configuration, save_path = save_path): # save all the configurations for each step of the algorithm
    
    file_name = "Algoritmo_di_ricerca_output_massimo_" + now.strftime("%d_%m_%Y_%H_%M_%S")+ device + '_CONFIGURAZIONE_COMPLETA.txt' 
    completeName = os.path.join(save_path, file_name)
    
    # Build the header of the file as timestamp+configuration+annotation+voltage_parameters+description
    global timestamp
    configuration = '\n%Laser Wavelength:' + str(laser_wavelength) + '[nm]\n%Laser Power:' +str(laser_power) + '[mW]\n%Amplifier Gain:' \
                  + str(amplifier_gain) + '[dB]'
    annotation = '\n%TEST ALGORITMO DI RICERCA OUTPUT MASSIMO. INPUT 5 E OUTPUT 5. TENTATIVO DI MASSIMIZZARE OUTPUT 5.\n'

    voltage_parameters = '%' # voltage_parameters are voltage_start, voltage_stop, voltage_step, triangular
    for element in voltage_start:
        voltage_parameters += str(element) + '\t'
    voltage_parameters += '\n%'
    for element in voltage_stop:
        voltage_parameters += str(element) + '\t'
    voltage_parameters += '\n%'
    for element in voltage_step:
        voltage_parameters += str(element) + '\t'
    voltage_parameters += '\n%' + str(F_triangular) + '\n'

    description = '%Swipe sulla tensione con onda sinusoidale su un canale, canali rimanenti a 0 V.Ciascun canale pilota una termoresistenza.' + device
    header = timestamp+configuration+annotation+voltage_parameters+description
    
    # Write the header and the column headers to the file
    meas_file = open(completeName, "a")
    meas_file.write(header)
    meas_file.write("\n%ID\tF_triangular\tPico Output\tActive channel")
    for j in range(len(channels)):
        meas_file.write("\tChannel " +str(channels[j]) + " : Voltage [V]\tCurrent [mA]")
    meas_file.write("\n")
    
    # Write each element of the target configuration list, going newline 
    for k in system_configuration:
        for z in range(len(k)-1):
            meas_file.write(str(k[z]) + '\t')
        meas_file.write(str(k[-1]) + '\n')
    
    
def save_optimal_configuration(target_configuration, save_path = save_path): # save only the configurations referring to the optimized maxima
    # Set the right path and file
    #save_path = "G:/Shared drives/FMPS Archive/Projects/0_QPIXPAD/QPX4/Opt.Meas/"+ device_model +"/"
    file_name = "Algoritmo_di_ricerca_output_massimo_" + now.strftime("%d_%m_%Y_%H_%M_%S")+ device + '.txt' 
    completeName = os.path.join(save_path, file_name)

    # Build the header of the file as timestamp+configuration+annotation+voltage_parameters+description
    global timestamp
    configuration = '\n%Laser Wavelength:' + str(laser_wavelength) + '[nm]\n%Laser Power:' +str(laser_power) + '[mW]\n%Amplifier Gain:' \
              + str(amplifier_gain) + '[dB]'

    annotation = '\n%TEST ALGORITMO DI RICERCA OUTPUT MASSIMO. INPUT 5 E OUTPUT 5. TENTATIVO DI MASSIMIZZARE OUTPUT 5.\n'
    termistori = "%" + str(channels) + "\n" #1,4,6,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,25,26,27\n'
    header = '%'+timestamp+configuration+annotation+termistori+'\n'

    # Write the header and the column headers to the file
    config_file = open(completeName, "a")
    config_file.write(header)
    meas_file.write("%ID\tPico Output")
    for j in range(len(channels)):
        meas_file.write("\tChannel " + str(channels[j]) + " Voltage")
    meas_file.write("\n")
    config_file.close()
    config_file = open(completeName, "a") 
            
    # Write each element of the target configuration list, going newline 
    for k in target_configuration:
        for z in range(len(k)):
            if z < 2:
                config_file.write(str(k[z]) + '\t')
            else:
                for w in range(len(k[z])-1):
                    config_file.write(str(k[z][w]) + '\t')
        config_file.write(str(k[-1][-1]) + '\n')
    
    config_file.close()

save_whole_system_configuration(system_configuration)
save_optimal_configuration(target_configuration)

print("\nALL DONE")

INFO:root:Capturing at sample interval  2.500E+05 ns, with total sampling time of  2.500E+06 ns



PERFORMING TRY N.1 NOW 


DRIVING CHANNEL 8 NOW 


Voltage maximum for channel 8 found: 8.9999 V

Corresponding output from Picoscope: 1969.298 mV

DRIVING CHANNEL 18 NOW 


Voltage maximum for channel 18 found: 0.0 V

Corresponding output from Picoscope: 1979.797 mV

DRIVING CHANNEL 15 NOW 


Voltage maximum for channel 15 found: 6.4999 V

Corresponding output from Picoscope: 2190.008 mV

DRIVING CHANNEL 9 NOW 


Voltage maximum for channel 9 found: 0.0 V

Corresponding output from Picoscope: 2193.915 mV

DRIVING CHANNEL 20 NOW 


Voltage maximum for channel 20 found: 0.0 V

Corresponding output from Picoscope: 2193.915 mV

DRIVING CHANNEL 4 NOW 


Voltage maximum for channel 4 found: 0.0 V

Corresponding output from Picoscope: 2193.915 mV

DRIVING CHANNEL 21 NOW 


Voltage maximum for channel 21 found: 0.0 V

Corresponding output from Picoscope: 2193.915 mV

DRIVING CHANNEL 17 NOW 


Voltage maximum for channel 17 found: 0.0 V

Corresponding output from Picoscope: 2193.915 mV

DRIVI

In [ ]:
# Set all channels to zero
q.v[:] = 0
q.i[:] = 0

# Close the communication with the driver
q.close()

%matplotlib widget                                       
axis_color = 'lightgoldenrodyellow'

hf_general = plt.figure()
hf_general.set_size_inches(15, 8, forward=True)
ha_general = hf_general.add_subplot(121)      # Output intensities (simulated)

# Adjust the subplots region to leave some space for the sliders and buttons
hf_general.subplots_adjust(bottom=0.30)
ha_general.set_title('Channel Voltage vs  Channel Current ')
ha_general.set_xlabel('Voltage [V]')
ha_general.set_ylabel('Current [mA]')
ha_general.set_xlim([-0.1, max(voltage_stop)])
ha_general.set_ylim([-0.1, 20])
ha_general.grid()
plt.rc('axes', prop_cycle=(cycler('color', ['r', 'g', 'b', 'c', 'm', 'y', 'k'])))



# Draw the initial plot(s) and text(s)
# The 'line' variable is used for modifying the line later
lines = []
z = 0

for x in range(7):
    if x == 4:
        z = 1
    [line] = ha_general.plot(0, 0, marker='o', label='T' + str(20-x-z) + 'T16 - CH' + str(3+x))
    lines.append(line)

ha_general.legend()  
    
locator = MaxNLocator(nbins=6)

F_overlapping_plots = 1

"""
Channel     Resistance
3           T20T16
4           T19T16
5           T18T16
6           T17T16
7           T15T16
8           T14T16
9           T13T16
"""

T13T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='T13T16 - CH8',
    continuous_update=True)  

T14T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='T14T16 - CH8',
    continuous_update=True) 

T15T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='T15T16 - CH8',
    continuous_update=True) 

T17T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='CHANNEL 6, MZI',
    continuous_update=True) 

T18T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='T18T16 - CH8',
    continuous_update=True) 

T19T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='T19T16 - CH8',
    continuous_update=True) 

T20T16_slider = widgets.FloatSlider(
    value=1, 
    min=0, max=voltage, step=0.1,
    description='T20T16 - CH8',
    continuous_update=True) 

def plot(T13T16_slider, T14T16_slider, T15T16_slider, T17T16_slider, T18T16_slider, T19T16_slider, T20T16_slider):
    
    q.v[3] = float(T20T16_slider)
    q.v[4] = float(T19T16_slider)
    q.v[5] = float(T18T16_slider)
    q.v[6] = float(T17T16_slider)
    q.v[7] = float(T15T16_slider)
    q.v[8] = float(T14T16_slider)
    q.v[9] = float(T13T16_slider)
    
    for x in range(7):
        temp_voltage = q.v[x+3]
        temp_current = q.i[x+3]
        lines[x].set_xdata(temp_voltage)
        lines[x].set_ydata(temp_current)
    hf_general.canvas.draw() 

z = interactive(plot, T13T16_slider=T13T16_slider, T14T16_slider=T14T16_slider, T15T16_slider=T15T16_slider, T17T16_slider=T17T16_slider, \
               T18T16_slider=T18T16_slider, T19T16_slider=T19T16_slider, T20T16_slider=T20T16_slider)
display(z)

plt.show()

In [ ]:
channels = [4,8,9,17,18,15,19,20,21]
total_steps = 10
system_configuration = [[] for _ in range(total_steps)]

ID = ""

for i in range(len(str(total_steps))):
    ID = ID + "0"
    
F_triangular = 1
valore_output_pico = 1999.25

# Creating a fixed length list for first row (for higher performance)
new_row = [-1 for _ in range((len(channels)*2)+4)]

# Populate first part of the new row with: ID, flag for triangular sweep, voltage value from pico, active channel
new_row[0] = int(ID)
new_row[1] = F_triangular
new_row[2] = valore_output_pico
new_row[3] = channels[0]

system_configuration[0] = new_row

print(system_configuration)
print(system_configuration[0])

In [ ]:
ID = 15
save_path = "C:/Users/lucag/Desktop"
mycustommodule.take_screenshot(image_name = str(ID), save_path = save_path)

ID = 150
# Take a screenshot of a resolution of screen_resolution and cutting everything that is on the left of xsrc (in pixels) and above ysrc
mycustommodule.take_screenshot(xdest= 0, ydest = 0, xsrc = 960, ysrc = 0, image_name = str(ID), save_path = save_path, screen_resolution = [960,1080])